In [23]:
import pandas as pd
import numpy as np

# 데이터 로드
flood_data = pd.read_excel('seoul_flood_data_with_drainage.xlsx')
rain_data = pd.read_csv('서울시날씨데이터.csv')

# 각 형식에 맞게 변환하기 위한 함수 정의
def parse_dates(date_str, time_str):
    if len(time_str) <= 2:
        combined = date_str + ' ' + time_str.zfill(2) + '00'
    else:
        combined = date_str + ' ' + time_str
    
    try:
        return pd.to_datetime(combined, format='%Y-%m-%d %H%M')
    except ValueError:
        try:
            return pd.to_datetime(combined, format='%Y%m%d %H%M')
        except ValueError:
            return pd.NaT

# '침수 시작 시간' 열에 변환된 datetime 값 할당
flood_data['침수 시작 시간'] = flood_data.apply(lambda row: parse_dates(str(row['침수 시작 날짜']), str(row['침수 시작 시간'])), axis=1)

# 강수 데이터의 날짜를 datetime으로 변환
rain_data['일시'] = pd.to_datetime(rain_data['일시'])

# 계산할 시간 범위 설정 (1시간부터 12시간까지)
time_ranges = range(1, 13)

# 각 구별로 강수량 계산 및 데이터 추가
for index, row in flood_data.iterrows():
    district = row['GU_NAM']
    flood_start = row['침수 시작 시간']
    extended_flood_time = flood_start + pd.Timedelta(hours=24)  # 침수 시작 시간에 3시간을 더함
    
    # 해당 구의 데이터만 필터링
    district_rain_data = rain_data[rain_data['지점'] == district]
    
    # 침수 시작 시간 ~ 3시간 이후까지의 데이터를 필터링
    filtered_data = district_rain_data[district_rain_data['일시'] <= extended_flood_time].sort_values(by='일시', ascending=False)
    
    for hours in time_ranges:
        # 특정 시간 범위의 데이터 필터링
        time_window = filtered_data[(filtered_data['일시'] > extended_flood_time - pd.Timedelta(hours=hours)) & (filtered_data['일시'] <= extended_flood_time)]
        
        if not time_window.empty:
            total_rainfall = time_window['강수량(mm)'].sum()
            avg_rainfall_per_10min = total_rainfall / (hours * 6)  # 1시간 = 60분, 10분 단위이므로 시간당 6개의 데이터가 존재함
            hourly_rainfall = time_window['강수량(mm)'].mean()
            
            # 결과를 데이터프레임에 추가
            flood_data.loc[index, f'{hours}시간 누적 강수량(mm)'] = total_rainfall
            flood_data.loc[index, f'{hours}시간 10분 평균 강수량(mm)'] = avg_rainfall_per_10min
            flood_data.loc[index, f'{hours}시간 시간당 강수량(mm)'] = hourly_rainfall
        else:
            # 데이터가 부족할 경우 NaN으로 채움
            flood_data.loc[index, f'{hours}시간 누적 강수량(mm)'] = np.nan
            flood_data.loc[index, f'{hours}시간 10분 평균 강수량(mm)'] = np.nan
            flood_data.loc[index, f'{hours}시간 시간당 강수량(mm)'] = np.nan

# 결과를 CSV 파일로 저장
output_csv_path = 'updated_seoul_flood_data.csv'
flood_data.to_csv(output_csv_path, index=False)

# CSV 파일 경로를 사용자에게 전달
output_csv_path


'updated_seoul_flood_data.csv'

In [24]:

# 강수량 관련 칼럼들
rainfall_columns = [
    f'{hours}시간 누적 강수량(mm)' for hours in time_ranges
] + [
    f'{hours}시간 10분 평균 강수량(mm)' for hours in time_ranges
] + [
    f'{hours}시간 시간당 강수량(mm)' for hours in time_ranges
]

# 전부 NaN이거나 비어있거나 0인 행 제거
flood_data = flood_data.dropna(subset=rainfall_columns, how='all')  # NaN인 행 제거
flood_data = flood_data[~(flood_data[rainfall_columns] == 0).all(axis=1)]  # 0인 행 제거
flood_data = flood_data[~(flood_data[rainfall_columns] == '').all(axis=1)]  # 비어있는 행 제거

# 결과를 CSV 파일로 저장
output_csv_path = 'updated_seoul_flood_data2.csv'
flood_data.to_csv(output_csv_path, index=False)

# CSV 파일 경로를 사용자에게 전달
output_csv_path


'updated_seoul_flood_data2.csv'

In [26]:

# 특정 강수량 관련 칼럼들
rainfall_columns = [
    '1시간 누적 강수량(mm)', '1시간 10분 평균 강수량(mm)', '1시간 시간당 강수량(mm)',
    '2시간 누적 강수량(mm)', '2시간 10분 평균 강수량(mm)', '2시간 시간당 강수량(mm)',
    '3시간 누적 강수량(mm)', '3시간 10분 평균 강수량(mm)', '3시간 시간당 강수량(mm)',
    '4시간 누적 강수량(mm)', '4시간 10분 평균 강수량(mm)', '4시간 시간당 강수량(mm)'
]

# 모든 값이 NaN이거나 비어있거나 0인 행 제거
condition = (flood_data[rainfall_columns].isna().all(axis=1)) | (flood_data[rainfall_columns] == 0).all(axis=1) | (flood_data[rainfall_columns] == '').all(axis=1)
flood_data = flood_data[~condition]

# 결과를 CSV 파일로 저장
output_csv_path = 'updated_seoul_flood_data.csv'
flood_data.to_csv(output_csv_path, index=False)

# CSV 파일 경로를 사용자에게 전달
output_csv_path

'updated_seoul_flood_data.csv'

In [29]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# 데이터 로드
flood_data = pd.read_csv('침수학습데이터.csv')
rain_data = pd.read_csv('서울시날씨데이터.csv')

# '일시' 열을 datetime 형식으로 변환
rain_data['일시'] = pd.to_datetime(rain_data['일시'])

# 기존의 침수 일자 리스트 생성
existing_flood_dates = flood_data['침수 시작 날짜'].unique()

# 랜덤 날짜/시간 생성 함수 정의 (2022년 중, 기존의 침수 일자와 겹치지 않도록)
def generate_random_datetime_in_2022(exclude_dates):
    start_date = datetime(2022, 1, 1)
    end_date = datetime(2022, 12, 31, 23, 59)
    while True:
        random_date = start_date + timedelta(seconds=random.randint(0, int((end_date - start_date).total_seconds())))
        if random_date.date() not in exclude_dates:
            return random_date

# 필요한 컬럼을 유지하면서 새로운 데이터 생성
new_data = flood_data[['centroid_lon', 'centroid_lat', 'GU_NAM', '배수정보', '침수된 지역의 평균 지형 고도']].copy()

# 랜덤 날짜와 시간을 생성하여 데이터프레임에 추가
new_data['침수 시작 날짜'] = new_data.apply(lambda _: generate_random_datetime_in_2022(existing_flood_dates).date(), axis=1)
new_data['침수 시작 시간'] = new_data.apply(lambda row: generate_random_datetime_in_2022(existing_flood_dates).time(), axis=1)

# '침수 시작 시간' 열을 datetime으로 변환
new_data['침수 시작 시간'] = pd.to_datetime(new_data['침수 시작 날짜'].astype(str) + ' ' + new_data['침수 시작 시간'].astype(str))

# 계산할 시간 범위 설정 (1시간부터 12시간까지)
time_ranges = range(1, 13)

# 각 구별로 강수량 계산 및 데이터 추가
for index, row in new_data.iterrows():
    district = row['GU_NAM']
    flood_start = row['침수 시작 시간']
    
    # 해당 구의 데이터만 필터링
    district_rain_data = rain_data[rain_data['지점'] == district]
    
    # 침수 시작 시간 이후의 데이터를 필터링
    filtered_data = district_rain_data[district_rain_data['일시'] >= flood_start].sort_values(by='일시', ascending=True)
    
    for hours in time_ranges:
        # 특정 시간 범위의 데이터 필터링
        time_window = filtered_data[(filtered_data['일시'] < flood_start + pd.Timedelta(hours=hours))]
        
        if not time_window.empty:
            total_rainfall = time_window['강수량(mm)'].sum()
            avg_rainfall_per_10min = total_rainfall / (hours * 6)  # 1시간 = 60분, 10분 단위이므로 시간당 6개의 데이터가 존재함
            hourly_rainfall = time_window['강수량(mm)'].mean()
            
            # 결과를 데이터프레임에 추가
            new_data.loc[index, f'{hours}시간 누적 강수량(mm)'] = total_rainfall
            new_data.loc[index, f'{hours}시간 10분 평균 강수량(mm)'] = avg_rainfall_per_10min
            new_data.loc[index, f'{hours}시간 시간당 강수량(mm)'] = hourly_rainfall
        else:
            # 데이터가 부족할 경우 NaN으로 채움
            new_data.loc[index, f'{hours}시간 누적 강수량(mm)'] = np.nan
            new_data.loc[index, f'{hours}시간 10분 평균 강수량(mm)'] = np.nan
            new_data.loc[index, f'{hours}시간 시간당 강수량(mm)'] = np.nan

# 결과를 CSV 파일로 저장
output_csv_path = 'updated_seoul_flood_data3.csv'
new_data.to_csv(output_csv_path, index=False)

# CSV 파일 경로를 사용자에게 전달
output_csv_path



'updated_seoul_flood_data3.csv'

In [32]:
# 특정 강수량 관련 칼럼들
rainfall_columns = [
    '1시간 누적 강수량(mm)', '1시간 10분 평균 강수량(mm)', '1시간 시간당 강수량(mm)',
    '2시간 누적 강수량(mm)', '2시간 10분 평균 강수량(mm)', '2시간 시간당 강수량(mm)',
    '3시간 누적 강수량(mm)', '3시간 10분 평균 강수량(mm)', '3시간 시간당 강수량(mm)',
    '4시간 누적 강수량(mm)', '4시간 10분 평균 강수량(mm)', '4시간 시간당 강수량(mm)'
]

# 모든 값이 NaN이거나 비어있거나 0인 행 제거
condition = (new_data[rainfall_columns].isna().all(axis=1)) | (new_data[rainfall_columns] == '').all(axis=1)
new_data = new_data[~condition]

# 결과를 CSV 파일로 저장
output_csv_path = 'updated_seoul_flood_data4.csv'
new_data.to_csv(output_csv_path, index=False)

# CSV 파일 경로를 사용자에게 전달
output_csv_path

'updated_seoul_flood_data4.csv'